<a href="https://colab.research.google.com/github/hanisdhuzuki/PETRONAS-Stock-Market-Behaviour/blob/main/PETGAS_Markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import io, base64, os, json, re 
import pandas as pd
import numpy as np
import datetime
from random import randint

In [ ]:
## load market data from Yahoo Finance 

In [ ]:
pnagf_df = pd.read_csv('PNAGF.csv')
pnagf_df['Date'] = pd.to_datetime(pnagf_df['Date'])

cut_off_date = '2017-05-27' 
pnagf_df = pnagf_df[pnagf_df['Date'] >= cut_off_date]


pnagf_df.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-05-30,5.25,5.25,5.25,5.25,2.633264,0
1,2017-05-31,5.25,5.25,5.25,5.25,2.633264,0
2,2017-06-01,5.25,5.25,5.25,5.25,2.633264,0
3,2017-06-02,5.25,5.25,5.25,5.25,2.633264,0
4,2017-06-05,5.25,5.25,5.25,5.25,2.633264,0


In [ ]:
# take random sets of sequential rows 
new_set = []
for row_set in range(0, 100000):
    if row_set%2000==0: print(row_set)
    row_quant = randint(10, 30)
    row_start = randint(0, len(pnagf_df)-row_quant)
    market_subset = pnagf_df.iloc[row_start:row_start+row_quant]

    Close_Date = max(market_subset['Date'])
    if row_set%2000==0: print(Close_Date)
    
    # Close_Gap = (market_subset['Close'] - market_subset['Close'].shift(1)) / market_subset['Close'].shift(1)
    Close_Gap = market_subset['Close'].pct_change()
    High_Gap = market_subset['High'].pct_change()
    Low_Gap = market_subset['Low'].pct_change() 
    Volume_Gap = market_subset['Volume'].pct_change() 
    Daily_Change = (market_subset['Close'] - market_subset['Open']) / market_subset['Open']
    Outcome_Next_Day_Direction = (market_subset['Volume'].shift(-1) - market_subset['Volume'])
    
    new_set.append(pd.DataFrame({'Sequence_ID':[row_set]*len(market_subset),
                            'Close_Date':[Close_Date]*len(market_subset),
                           'Close_Gap':Close_Gap,
                           'High_Gap':High_Gap,
                           'Low_Gap':Low_Gap,
                           'Volume_Gap':Volume_Gap,
                           'Daily_Change':Daily_Change,
                           'Outcome_Next_Day_Direction':Outcome_Next_Day_Direction}))

 

0
2019-06-18 00:00:00
2000
2019-04-02 00:00:00
4000
2021-05-11 00:00:00
6000
2017-08-30 00:00:00
8000
2020-02-14 00:00:00
10000
2021-09-13 00:00:00
12000
2022-05-23 00:00:00
14000
2020-12-22 00:00:00
16000
2020-09-02 00:00:00
18000
2018-12-10 00:00:00
20000
2020-03-27 00:00:00
22000
2021-03-01 00:00:00
24000
2020-01-23 00:00:00
26000
2021-06-22 00:00:00
28000
2020-02-18 00:00:00
30000
2020-10-12 00:00:00
32000
2019-11-27 00:00:00
34000
2018-09-21 00:00:00
36000
2020-07-14 00:00:00
38000
2021-02-22 00:00:00
40000
2017-08-10 00:00:00
42000
2019-06-27 00:00:00
44000
2021-09-16 00:00:00
46000
2019-12-02 00:00:00
48000
2021-04-28 00:00:00
50000
2018-11-15 00:00:00
52000
2018-01-30 00:00:00
54000
2018-10-29 00:00:00
56000
2022-04-14 00:00:00
58000
2018-12-28 00:00:00
60000
2021-09-10 00:00:00
62000
2018-10-16 00:00:00
64000
2019-01-22 00:00:00
66000
2017-11-10 00:00:00
68000
2021-04-15 00:00:00
70000
2022-01-13 00:00:00
72000
2021-06-02 00:00:00
74000
2019-10-24 00:00:00
76000
2018-04-30 00:

In [ ]:
len(market_subset)

18

In [ ]:
new_set_df = pd.concat(new_set)
print(new_set_df.shape)
new_set_df = new_set_df.dropna(how='any') 
print(new_set_df.shape)
new_set_df.tail(20)

(2002624, 8)
(26435, 8)


,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
183,99938,2018-03-07,0.186869,0.186869,0.186869,inf,0.0,-445.0
184,99938,2018-03-07,0.000000,0.000000,0.000000,-1.0,0.0,0.0
190,99938,2018-03-07,-0.021277,-0.021277,-0.021277,inf,0.0,-334.0
191,99938,2018-03-07,0.000000,0.000000,0.000000,-1.0,0.0,0.0
183,99951,2018-03-27,0.186869,0.186869,0.186869,inf,0.0,-445.0
184,99951,2018-03-27,0.000000,0.000000,0.000000,-1.0,0.0,0.0
190,99951,2018-03-27,-0.021277,-0.021277,-0.021277,inf,0.0,-334.0
191,99951,2018-03-27,0.000000,0.000000,0.000000,-1.0,0.0,0.0
190,99954,2018-03-20,-0.021277,-0.021277,-0.021277,inf,0.0,-334.0
191,99954,2018-03-20,0.000000,0.000000,0.000000,-1.0,0.0,0.0


In [ ]:
new_set_df.head()

,Sequence_ID,Close_Date,Close_Gap,High_Gap,Low_Gap,Volume_Gap,Daily_Change,Outcome_Next_Day_Direction
190,4,2018-04-10,-0.021277,-0.021277,-0.021277,inf,0.0,-334.0
191,4,2018-04-10,0.000000,0.000000,0.000000,-1.0,0.0,0.0
955,14,2021-03-26,-0.094437,-0.094437,-0.094437,inf,0.0,-700.0
956,14,2021-03-26,0.000000,0.000000,0.000000,-1.0,0.0,0.0
955,34,2021-04-01,-0.094437,-0.094437,-0.094437,inf,0.0,-700.0


In [ ]:
# confirm sequence
# new_set_df[new_set_df['Close_Date'] == '2022-05-27'] {HLH, HLH, HHH, HHH, LLL, LML, LML, LLL, LHL...}

In [ ]:
# create sequences
# simplify the data by binning values into three groups

# Close_Gap
new_set_df['Close_Gap_LMH'] = pd.qcut(new_set_df['Close_Gap'], 3, labels=["L", "M", "H"])

# Volume_Gap
new_set_df['Volume_Gap_LMH'] = pd.qcut(new_set_df['Volume_Gap'], 3, labels=["L", "M", "H"])
 
# Daily_Change
new_set_df['Daily_Change_LMH'] = pd.qcut(new_set_df['Daily_Change'], 3, labels=["L", "M", "H"])

# new set
new_set_df = new_set_df[["Sequence_ID", 
                         "Close_Date", 
                         "Close_Gap_LMH", 
                         "Volume_Gap_LMH", 
                         "Daily_Change_LMH", 
                         "Outcome_Next_Day_Direction"]]

new_set_df['Event_Pattern'] = new_set_df['Close_Gap_LMH'].astype(str) + new_set_df['Volume_Gap_LMH'].astype(str) + new_set_df['Daily_Change_LMH'].astype(str)

/usr/local/lib/python3.7/dist-packages/pandas/core/algorithms.py:1167: RuntimeWarning: invalid value encountered in double_scalars
  return a + (b - a) * fraction


ValueError: ignored

In [ ]:
new_set_df.tail(10)

In [ ]:
new_set_df['Outcome_Next_Day_Direction'].describe()

In [ ]:
# reduce the set
compressed_set = new_set_df.groupby(['Sequence_ID', 
                                     'Close_Date'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index()

print(compressed_set.shape)
compressed_set.head() 

In [ ]:
#compressed_outcomes = new_set_df[['Sequence_ID', 'Close_Date', 'Outcome_Next_Day_Direction']].groupby(['Sequence_ID', 'Close_Date']).agg()

compressed_outcomes = new_set_df.groupby(['Sequence_ID', 'Close_Date'])['Outcome_Next_Day_Direction'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()
print(compressed_outcomes.shape)
compressed_outcomes.describe()

In [ ]:
compressed_set = pd.merge(compressed_set, compressed_outcomes, on= ['Sequence_ID', 'Close_Date'], how='inner')
print(compressed_set.shape)
compressed_set.head()

In [ ]:
# # reduce set 

# compressed_set = new_set_df.groupby(['Sequence_ID', 'Close_Date','Outcome_Next_Day_Direction'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index()

compressed_set['Event_Pattern'] = [''.join(e.split()).replace('{','')
                                   .replace('}','') for e in compressed_set['Event_Pattern'].values]
compressed_set.head()

In [ ]:
# use last x days of data for validation
compressed_set_validation = compressed_set[compressed_set['Close_Date'] >= datetime.datetime.now() 
                                           - datetime.timedelta(days=90)] # Sys.Date()-90 

compressed_set_validation.shape

In [ ]:
compressed_set = compressed_set[compressed_set['Close_Date'] < datetime.datetime.now() 
                                           - datetime.timedelta(days=90)]  
compressed_set.shape

In [ ]:
list(compressed_set)

In [ ]:
# drop date field
compressed_set = compressed_set[['Sequence_ID', 'Event_Pattern','Outcome_Next_Day_Direction']]
compressed_set_validation = compressed_set_validation[['Sequence_ID', 'Event_Pattern','Outcome_Next_Day_Direction']]

In [ ]:
compressed_set['Outcome_Next_Day_Direction'].describe()

In [ ]:
print(len(compressed_set['Outcome_Next_Day_Direction']))
len(compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000])

In [ ]:
# keep only keep big/interesting moves 
print('all moves:', len(compressed_set))
compressed_set = compressed_set[abs(compressed_set['Outcome_Next_Day_Direction']) > 10000000]
compressed_set['Outcome_Next_Day_Direction'] = np.where((compressed_set['Outcome_Next_Day_Direction'] > 0), 1, 0)
compressed_set_validation['Outcome_Next_Day_Direction'] = np.where((compressed_set_validation['Outcome_Next_Day_Direction'] > 0), 1, 0)
print('big moves only:', len(compressed_set))            


In [ ]:
compressed_set.head()

In [ ]:
# create two data sets - won/not won
compressed_set_pos = compressed_set[compressed_set['Outcome_Next_Day_Direction']==1][['Sequence_ID', 'Event_Pattern']]
print(compressed_set_pos.shape)
compressed_set_neg = compressed_set[compressed_set['Outcome_Next_Day_Direction']==0][['Sequence_ID', 'Event_Pattern']]
print(compressed_set_neg.shape)

In [ ]:
flat_list = [item.split(',') for item in compressed_set['Event_Pattern'].values ]
unique_patterns = ','.join(str(r) for v in flat_list for r in v)
unique_patterns = list(set(unique_patterns.split(',')))
len(unique_patterns)

In [ ]:
compressed_set['Outcome_Next_Day_Direction'].head() 

In [ ]:
# build the markov transition grid
def build_transition_grid(compressed_grid, unique_patterns):
    # build the markov transition grid

    patterns = []
    counts = []
    for from_event in unique_patterns:

        # how many times 
        for to_event in unique_patterns:
            pattern = from_event + ',' + to_event # MMM,MlM

            ids_matches = compressed_grid[compressed_grid['Event_Pattern'].str.contains(pattern)]
            found = 0
            if len(ids_matches) > 0:
                Event_Pattern = '---'.join(ids_matches['Event_Pattern'].values)
                found = Event_Pattern.count(pattern)
            patterns.append(pattern)
            counts.append(found)

    # create to/from grid
    grid_Df = pd.DataFrame({'pairs':patterns, 'counts': counts})

    grid_Df['x'], grid_Df['y'] = grid_Df['pairs'].str.split(',', 1).str
    grid_Df.head()

    grid_Df = grid_Df.pivot(index='x', columns='y', values='counts')

    grid_Df.columns= [col for col in grid_Df.columns]
    del grid_Df.index.name

    # replace all NaN with zeros
    grid_Df.fillna(0, inplace=True)
    grid_Df.head()

    #grid_Df.rowSums(transition_dataframe) 
    grid_Df = grid_Df / grid_Df.sum(1)
    return (grid_Df)

In [ ]:
grid_pos = build_transition_grid(compressed_set_pos, unique_patterns) 
grid_neg = build_transition_grid(compressed_set_neg, unique_patterns)

In [ ]:
grid_neg.head()

In [ ]:
grid_pos.head()

In [ ]:
# compressed_set_validation[compressed_set_validation['Sequence_ID' == seq_id]]

In [ ]:
def safe_log(x,y):
   try:
      lg = np.log(x/y)
   except:
      lg = 0
   return lg

# predict on out of sample data
actual = []
predicted = []
for seq_id in compressed_set_validation['Sequence_ID'].values:
    patterns = compressed_set_validation[compressed_set_validation['Sequence_ID'] == seq_id]['Event_Pattern'].values[0].split(',')
    pos = []
    neg = []
    log_odds = []
    
    for id in range(0, len(patterns)-1):
        # get log odds
        # logOdds = log(tp(i,j) / tn(i,j)
        if (patterns[id] in list(grid_pos) and patterns[id+1] in list(grid_pos) and patterns[id] in list(grid_neg) and patterns[id+1] in list(grid_neg)):
                
            numerator = grid_pos[patterns[id]][patterns[id+1]]
            denominator = grid_neg[patterns[id]][patterns[id+1]]
            if (numerator == 0 and denominator == 0):
                log_value =0
            elif (denominator == 0):
                log_value = np.log(numerator / 0.00001)
            elif (numerator == 0):
                log_value = np.log(0.00001 / denominator)
            else:
                log_value = np.log(numerator/denominator)
        else:
            log_value = 0
        
        log_odds.append(log_value)
        
        pos.append(numerator)
        neg.append(denominator)
      
    print('outcome:', compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    print(sum(pos)/sum(neg))
    print(sum(log_odds))

    actual.append(compressed_set_validation[compressed_set_validation['Sequence_ID']==seq_id]['Outcome_Next_Day_Direction'].values[0])
    predicted.append(sum(log_odds))

from sklearn.metrics import confusion_matrix

confusion_matrix(actual, [1 if p > 0 else 0 for p in predicted])

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(actual, [1 if p > 0 else 0 for p in predicted])
print('Accuracy:', round(score * 100,2), '%')

In [ ]:
import seaborn as sns
cm = confusion_matrix(actual, [1 if p > 0 else 0 for p in predicted])
fig, ax = plt.subplots(figsize=(5, 5))
sns.heatmap(cm, annot=True, ax = ax, fmt='g')

ax.set_title('Confusion Matrix') 
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')

ax.xaxis.set_ticklabels(['up day','down day'])
ax.yaxis.set_ticklabels(['up day','down day'])
ax.set_yticklabels(ax.get_yticklabels(), rotation = 0, fontsize = 8)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90, fontsize = 8)  
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')